In [1]:
import nltk
import pandas as pd
nltk.download('popular', quiet=True)
import demoji
from wordcloud import WordCloud
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import bigrams
from nltk import FreqDist
import spacy
import string
nltk.download("wordnet", quiet=True)
nltk.download("stopwords", quiet=True)
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from collections import defaultdict
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [2]:
import os
import glob
def collect_data():
    text_file_pattern = "*.txt"  # You can adjust the pattern to match your file extensions
    text_files = glob.glob(os.path.join("../nhs/content", text_file_pattern))
    data = {}
    for file_path in text_files:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_name = os.path.basename(file_path)
            file_content = file.read()
            data[file_name] = file_content
    return data

In [3]:
corpus = collect_data()
text = ""
for data in corpus:
    text += " " + data

In [7]:
def remove_punctuation(text):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    # Use translate method to remove punctuation
    cleaned_text = text.translate(translator)
    return cleaned_text

def remove_stop_words(text):
    nltk_stopwords = stopwords.words('english')
    spacy_stopwords = nlp.Defaults.stop_words

    stop_words = (*nltk_stopwords, *spacy_stopwords, "NHStxt")

    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return " ".join(tokens)

def apply_lemmitization(text):
    tag_map = defaultdict(lambda : wordnet.NOUN)
    tag_map['V'] = wordnet.VERB
    tag_map['A'] = wordnet.ADJ
    tag_map['R'] = wordnet.ADJ

    lemmitizer = WordNetLemmatizer()
    lemmitized_result = ""
    tokens = word_tokenize(text)
    for token, tag in pos_tag(tokens):
        lemma = lemmitizer.lemmatize(token, tag_map[tag[0]])
        lemmitized_result = lemmitized_result + " " + lemma
    return lemmitized_result

def remove_emoji_and_smart_quotes(text):
    # replacing emojis with description
    text = demoji.replace_with_desc(text)
    #Removing smart quotes
    return text.replace(""", "\"").replace(""","\"")

## 1. Perform necessary data preprocessing, e.g. removing punctuation and stop words, stemming, lemmatizing. You may use the outputs from previous weekly assignments. (10 points)


In [9]:
def data_preprocessing(text):
    text = remove_emoji_and_smart_quotes(text)
    text = remove_punctuation(text)
    text = remove_stop_words(text)
    text = apply_lemmitization(text)
    return text

def apply_data_preprocessing_to_corpus(corpus):
    new_corpus = {}
    for idx, key in enumerate(corpus.keys()):
        new_corpus[key] = data_preprocessing(corpus[key])
        print(f"idx: {idx}")
    return new_corpus

In [63]:
processed_text = data_preprocessing(text)
with open('week7_1.txt', 'w') as file:
    file.write(f'{processed_text}')

## 2. Count BoW on pre-processed data.  (10 points)


In [11]:

def get_document_features(word_features, text):
    features = {}
    tokens = word_tokenize(text)
    for word in word_features:
        features[word] = tokens.count(word)
    return features

def get_corpus_features(corpus, text):
    text = text.lower()
    words = word_tokenize(text)
    word_features = list(FreqDist(words).most_common(1000))
    word_features = [i for i,j in word_features]
    dataframe_data = []
    for key in corpus.keys():
        features = get_document_features(word_features, corpus[key])
        features['title'] = key
        dataframe_data.append(features)
    return dataframe_data


In [65]:
df_data = get_corpus_features(corpus, processed_text)
df = pd.DataFrame(df_data)
df

,syndrome,disease,pain,cancer,disorder,test,blood,eye,child,foot,...,bedbugs,versicolor,pilonidal,system,labyrinthitis,neuritis,tendon,clinical,trial,title
0,0,0,0,0,1,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,Food colours and hyperactivity NHS.txt
1,0,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lung health checks NHS.txt
2,0,2,0,0,0,0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,Carotid endarterectomy NHS.txt
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Middle East respiratory syndrome MERS NHS.txt
4,0,0,2,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,Kidney infection NHS.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,0,0,3,0,0,1,1,0,1,1,...,0,0,0,1,0,0,0,0,0,Osteomyelitis NHS.txt
974,0,0,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,Floaters and flashes in the eyes NHS.txt
975,0,16,0,0,0,1,1,0,12,0,...,0,0,0,0,0,0,0,1,0,TaySachs disease NHS.txt
976,11,2,3,0,1,0,2,0,2,0,...,0,0,0,2,0,0,0,0,0,Slapped cheek syndrome NHS.txt


## 3. Compute TF-IDF vectors on pre-processed data.  (20 points)


In [13]:
processed_corpus = apply_data_preprocessing_to_corpus(corpus)

,syndrome,disease,pain,cancer,disorder,test,blood,eye,child,foot,...,bedbugs,versicolor,pilonidal,system,labyrinthitis,neuritis,tendon,clinical,trial,title
0,0,0,0,0,1,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,Food colours and hyperactivity NHS.txt
1,0,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lung health checks NHS.txt
2,0,2,0,0,0,0,6,1,0,0,...,0,0,0,0,0,0,0,0,0,Carotid endarterectomy NHS.txt
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Middle East respiratory syndrome MERS NHS.txt
4,0,0,2,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,Kidney infection NHS.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,0,0,3,0,0,1,1,0,1,1,...,0,0,0,1,0,0,0,0,0,Osteomyelitis NHS.txt
974,0,0,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,Floaters and flashes in the eyes NHS.txt
975,0,16,0,0,0,1,1,0,12,0,...,0,0,0,0,0,0,0,1,0,TaySachs disease NHS.txt
976,11,2,3,0,1,0,2,0,2,0,...,0,0,0,2,0,0,0,0,0,Slapped cheek syndrome NHS.txt


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(text):
    return word_tokenize(text)

max_features = 200
tfidfVectorizer = TfidfVectorizer(input="content", use_idf=True,
                                  tokenizer=tokenize, max_features=max_features,
                                  stop_words="english")


tfidf = tfidfVectorizer.fit_transform(processed_corpus.values())

tfidf_tokens = tfidfVectorizer.get_feature_names_out()

final_vectors = pd.DataFrame(
    data = tfidf.toarray(),
    columns = tfidf_tokens
)
final_vectors


/Users/shireesh/opt/anaconda3/envs/COMP293/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,1,111,2,3,4,able,activity,adult,advice,affect,...,white,woman,work,worse,year,youll,youre,youve,–,’
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03637,0.00000,0.020419,0.022037,...,0.0,0.000000,0.000000,0.000000,0.000000,0.032358,0.000000,0.0,0.000000,0.000000
1,0.011828,0.000000,0.000000,0.013468,0.016859,0.000000,0.00000,0.00000,0.036549,0.000000,...,0.0,0.000000,0.012683,0.000000,0.000000,0.014480,0.056743,0.0,0.033590,0.000000
2,0.125821,0.000000,0.063157,0.143263,0.000000,0.000000,0.00000,0.00000,0.048598,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.077013,0.050299,0.0,0.089326,0.000000
3,0.000000,0.248377,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.066132,0.000000
4,0.000000,0.000000,0.016905,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.024549,0.018056,0.021522,0.000000,0.020614,0.053854,0.0,0.023910,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,0.033728,0.134896,0.000000,0.038403,0.048071,0.036776,0.00000,0.00000,0.026054,0.112475,...,0.0,0.000000,0.000000,0.000000,0.000000,0.041288,0.026966,0.0,0.000000,0.000000
974,0.000000,0.192250,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.037132,0.080148,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
975,0.000000,0.000000,0.061306,0.069532,0.087038,0.066586,0.00000,0.00000,0.094349,0.025456,...,0.0,0.000000,0.098217,0.039025,0.035993,0.000000,0.097651,0.0,0.108385,0.000000
976,0.031548,0.000000,0.063343,0.035921,0.000000,0.000000,0.00000,0.04648,0.024371,0.000000,...,0.0,0.000000,0.033827,0.000000,0.000000,0.000000,0.075672,0.0,0.111986,0.037131


## 4. Perform integer encoding and one-hot encoding on one of the pre-processed data files and save the output to a txt file.  (30 points)


In [15]:
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
keys = list(processed_corpus.keys())
tokens = tokenize(processed_corpus[keys[0]])
integer_encoded = label_encoder.fit_transform(tokens)
print(integer_encoded)

/Users/shireesh/opt/anaconda3/envs/COMP293/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,1,111,2,3,4,able,activity,adult,advice,affect,...,white,woman,work,worse,year,youll,youre,youve,–,’
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03637,0.00000,0.020419,0.022037,...,0.0,0.000000,0.000000,0.000000,0.000000,0.032358,0.000000,0.0,0.000000,0.000000
1,0.011828,0.000000,0.000000,0.013468,0.016859,0.000000,0.00000,0.00000,0.036549,0.000000,...,0.0,0.000000,0.012683,0.000000,0.000000,0.014480,0.056743,0.0,0.033590,0.000000
2,0.125821,0.000000,0.063157,0.143263,0.000000,0.000000,0.00000,0.00000,0.048598,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.077013,0.050299,0.0,0.089326,0.000000
3,0.000000,0.248377,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.066132,0.000000
4,0.000000,0.000000,0.016905,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.024549,0.018056,0.021522,0.000000,0.020614,0.053854,0.0,0.023910,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,0.033728,0.134896,0.000000,0.038403,0.048071,0.036776,0.00000,0.00000,0.026054,0.112475,...,0.0,0.000000,0.000000,0.000000,0.000000,0.041288,0.026966,0.0,0.000000,0.000000
974,0.000000,0.192250,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.037132,0.080148,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
975,0.000000,0.000000,0.061306,0.069532,0.087038,0.066586,0.00000,0.00000,0.094349,0.025456,...,0.0,0.000000,0.098217,0.039025,0.035993,0.000000,0.097651,0.0,0.108385,0.000000
976,0.031548,0.000000,0.063343,0.035921,0.000000,0.000000,0.00000,0.04648,0.024371,0.000000,...,0.0,0.000000,0.033827,0.000000,0.000000,0.000000,0.075672,0.0,0.111986,0.037131


In [86]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(
    len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(
    integer_encoded)
print(onehot_encoded)
pd.DataFrame(onehot_encoded).to_csv("week7_4.txt", sep='\t', index=False)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/Users/shireesh/opt/anaconda3/envs/COMP293/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [77]:
inverted = label_encoder.inverse_transform(
    [argmax(onehot_encoded[0, :])])
print(inverted)

['Research']


## 5. Choose an appropriate word and find the words that are the most similar to it in one of the pre-processed data files.  (30 points)


In [98]:
import gensim
import nltk
from nltk.corpus import movie_reviews
from gensim.models import Word2Vec
# make a list of movie review documents
documents = [text.split() for text in processed_corpus.values()]
model = Word2Vec(documents, min_count=5)
model.wv.most_similar(positive = ['cancer'],topn = 25)


[('breast', 0.9294400811195374),
 ('penile', 0.9016801714897156),
 ('nonmelanoma', 0.9012202024459839),
 ('testicular', 0.8954606056213379),
 ('nasopharyngeal', 0.8859248757362366),
 ('laryngeal', 0.8493385314941406),
 ('prostate', 0.8281572461128235),
 ('Prostate', 0.8127396106719971),
 ('ovarian', 0.7976941466331482),
 ('sinus', 0.7793684601783752),
 ('Types', 0.777524471282959),
 ('melanoma', 0.7684705853462219),
 ('vulval', 0.7663765549659729),
 ('polyp', 0.7647110223770142),
 ('bowel', 0.7627311944961548),
 ('tumour', 0.7616214752197266),
 ('Hodgkin', 0.7559468150138855),
 ('lymphoma', 0.7526421546936035),
 ('cancerous', 0.7447134256362915),
 ('nonHodgkin', 0.7429410815238953),
 ('advanced', 0.7410709857940674),
 ('squamous', 0.7305600643157959),
 ('myeloma', 0.7272151112556458),
 ('radiotherapy', 0.7244316339492798),
 ('pilonidal', 0.719566822052002)]